In [1]:
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas as pd
import sklearn
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import learning_curve
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV


from mlflow.tracking import MlflowClient
from urllib.parse import urlparse
from sklearn.svm import SVC
from sklearn.metrics._plot.precision_recall_curve import precision_recall_curve

In [28]:
!pip install matplotlib


In [29]:
!pip install textblob
!pip install pandas
!pip install scikit-learn
!pip install nltk


In [30]:
!pip install mlflow

In [9]:
import mlflow

**Importing train, validation and test set**

In [2]:
train_data = pd.read_csv("train.csv")
validation_data=pd.read_csv("test.csv")
test_data=pd.read_csv("validation.csv")

In [3]:
from sklearn.metrics import auc
def aucpr_socre(true_labels, predicted_labels):
    pre,re,_=precision_recall_curve(true_labels, predicted_labels)
    aucpr= auc(re,pre)
    return aucpr


**Assigning labels, vectorizing the text data, applying TF-IDF**

In [4]:
# Assign features and labels
X_train = train_data['processed_text']
y_train = train_data['spam']
X_val = validation_data['processed_text']
y_val = validation_data['spam']

X_test = test_data['processed_text']
y_test = test_data['spam']

X_train = X_train.fillna('')

# Vectorize the text data using CountVectorizer
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_val_counts = vectorizer.transform(X_val)
X_test_counts = vectorizer.transform(X_test)

# Apply TF-IDF transformation
tfidf_transformer = TfidfTransformer()
X_train_vec = tfidf_transformer.fit_transform(X_train_counts)
X_val_vec = tfidf_transformer.transform(X_val_counts)
X_test_vec = tfidf_transformer.transform(X_test_counts)




**Random Forest**

In [5]:
from sklearn.metrics import confusion_matrix

# Random Forest
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_vec, y_train)
rf_predictions = rf_classifier.predict(X_test_vec)

rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_precision = precision_score(y_test, rf_predictions)
rf_recall = recall_score(y_test, rf_predictions)

print("Random Forest Validation Metrics:")
print("Accuracy:", rf_accuracy)
print("Precision:", rf_precision)
print("Recall:", rf_recall)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, rf_predictions)
print("\nConfusion Matrix:")
print(conf_matrix)


Random Forest Validation Metrics:
Accuracy: 0.9890310786106032
Precision: 0.9894736842105263
Recall: 0.9894736842105263

Confusion Matrix:
[[259   3]
 [  3 282]]


**Hyperparameter Tuning**

In [6]:
rf_param_grid = {'n_estimators': [100, 200, 300],
                 'max_depth': [None, 10, 20]}

rf_grid_search = GridSearchCV(RandomForestClassifier(), rf_param_grid, cv=5, n_jobs=-1)
rf_grid_search.fit(X_val_vec, y_val)

print("\nRandom Forest Best Parameters:", rf_grid_search.best_params_)
print("Random Forest Best Score:", rf_grid_search.best_score_)


Random Forest Best Parameters: {'max_depth': None, 'n_estimators': 200}
Random Forest Best Score: 0.9617514595496248


**Fitting the best hyperparameters values and checking the performance of the model**

In [7]:
best_rf = RandomForestClassifier(n_estimators=rf_grid_search.best_params_['n_estimators'],
                                  max_depth=rf_grid_search.best_params_['max_depth'])

# Fit models to training data
best_rf.fit(X_train_vec, y_train)

# Predict on test set
rf_test_predictions = best_rf.predict(X_test_vec)
# Evaluate performance on test set
rf_test_accuracy = accuracy_score(y_test, rf_test_predictions)

rf_test_precision = precision_score(y_test, rf_test_predictions)
rf_test_recall = recall_score(y_test, rf_test_predictions)

print("Random Forest Validation Metrics:")
print("Accuracy:", rf_test_accuracy)
print("Precision:", rf_test_precision)
print("Recall:", rf_test_recall)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, rf_test_predictions)
print("\nConfusion Matrix:")
print(conf_matrix)


Random Forest Validation Metrics:
Accuracy: 0.9890310786106032
Precision: 0.9860627177700348
Recall: 0.9929824561403509

Confusion Matrix:
[[258   4]
 [  2 283]]


In [10]:

# Assuming X_train_vec, X_test_vec, y_train, y_test, eval_metrics are defined

with mlflow.start_run(nested=True):
    # Define and train the model
    best_rf = RandomForestClassifier(n_estimators=rf_grid_search.best_params_['n_estimators'],
                                  max_depth=rf_grid_search.best_params_['max_depth'])
    
    best_rf.fit(X_train_vec,y_train)

    # Predict on test set
    predicted_ = best_rf.predict(X_test_vec)

    # Evaluate the model
    aucpr = aucpr_socre(y_test, predicted_)

    # Log evaluation metric
    mlflow.log_metric("Area under Precision Recall Curve", aucpr)

    # Print evaluation metric
    print("Random Forest Classifier model (Area under Precision_Recall Curve={:.2f}):".format(aucpr))

    # Log the model
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != "file":
        # If not storing in a local file, register the model with a name
        mlflow.sklearn.log_model(best_rf, "model", registered_model_name="Random Forest Classifier")
    else:
        # If storing locally, simply log the model
        mlflow.sklearn.log_model(best_rf, "model")

    # End MLflow run
    mlflow.end_run()

Random Forest Classifier model (Area under Precision_Recall Curve=0.99):


**Logistic Regression**

In [11]:
# Logistic Regression
lr_classifier = LogisticRegression()
lr_classifier.fit(X_train_vec, y_train)
lr_predictions = lr_classifier.predict(X_test_vec)

lr_accuracy = accuracy_score(y_test, lr_predictions)
lr_precision = precision_score(y_test, lr_predictions)
lr_recall = recall_score(y_test, lr_predictions)

print("\nLogistic Regression Validation Metrics:")
print("Accuracy:", lr_accuracy)
print("Precision:", lr_precision)
print("Recall:", lr_recall)
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, lr_predictions)
print("\nConfusion Matrix:")
print(conf_matrix)



Logistic Regression Validation Metrics:
Accuracy: 0.9835466179159049
Precision: 0.9758620689655172
Recall: 0.9929824561403509

Confusion Matrix:
[[255   7]
 [  2 283]]


**Hyperparameter Tuning**

In [13]:
# Logistic Regression
lr_param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}

lr_grid_search = GridSearchCV(LogisticRegression(), lr_param_grid, cv=5, n_jobs=-1)
lr_grid_search.fit(X_val_vec, y_val)

print("\nLogistic Regression Best Parameters:", lr_grid_search.best_params_)
print("Logistic Regression Best Score:", lr_grid_search.best_score_)


Logistic Regression Best Parameters: {'C': 10}
Logistic Regression Best Score: 0.9817681401167639


**Fitting the best hyperparameters values and checking the performance of the model**

In [14]:
best_lr = LogisticRegression(C=lr_grid_search.best_params_['C'])

best_lr.fit(X_train_vec, y_train)
lr_test_predictions = best_lr.predict(X_test_vec)

lr_test_accuracy = accuracy_score(y_test, lr_test_predictions)
lr_test_precision = precision_score(y_test, lr_test_predictions)
lr_test_recall = recall_score(y_test, lr_test_predictions)

print("\nLogistic Regression Validation Metrics:")
print("Accuracy:", lr_test_accuracy)
print("Precision:", lr_test_precision)
print("Recall:", lr_test_recall)


# Confusion Matrix
conf_matrix = confusion_matrix(y_test, lr_test_predictions)
print("\nConfusion Matrix:")
print(conf_matrix)


Logistic Regression Validation Metrics:
Accuracy: 0.9853747714808044
Precision: 0.9792387543252595
Recall: 0.9929824561403509

Confusion Matrix:
[[256   6]
 [  2 283]]


In [16]:
# Assuming X_train_vec, X_test_vec, y_train, y_test, eval_metrics are defined

with mlflow.start_run():
    # Define and train the model
    best_lr = LogisticRegression(C=lr_grid_search.best_params_['C'])
    best_lr.fit(X_train_vec, y_train)

    # Predict on test set
    predicted_ = best_lr.predict(X_test_vec)

    # Evaluate the model
    aucpr = aucpr_socre(y_test, predicted_)

    # Log evaluation metric
    mlflow.log_metric("Area under Precision Recall Curve", aucpr)

    # Print evaluation metric
    print("Logistic Regression model (Area under Precision_Recall Curve={:.2f}):".format(aucpr))

    # Log the model
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != "file":
        # If not storing in a local file, register the model with a name
        mlflow.sklearn.log_model(best_lr, "model", registered_model_name="Logistic Regression")
    else:
        # If storing locally, simply log the model
        mlflow.sklearn.log_model(best_lr, "model")

    # End MLflow run
    mlflow.end_run()

Logistic Regression model (Area under Precision_Recall Curve=0.99):


C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2288.0_x64__qbz5n2kfra8p0\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2288.0_x64__qbz5n2kfra8p0\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


**Support Vector Machine**

In [18]:
# SVM
svm_classifier = SVC()
svm_classifier.fit(X_train_vec, y_train)
svm_predictions = svm_classifier.predict(X_test_vec)

svm_accuracy = accuracy_score(y_test, svm_predictions)
svm_precision = precision_score(y_test, svm_predictions)
svm_recall = recall_score(y_test, svm_predictions)

print("\nSVM Validation Metrics:")
print("Accuracy:", svm_accuracy)
print("Precision:", svm_precision)
print("Recall:", svm_recall)
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, svm_predictions)
print("\nConfusion Matrix:")
print(conf_matrix)


SVM Validation Metrics:
Accuracy: 0.9872029250457038
Precision: 0.9826388888888888
Recall: 0.9929824561403509

Confusion Matrix:
[[257   5]
 [  2 283]]


**Hyperparameter Tuning**

In [17]:
# SVM
svm_param_grid = {'C': [0.1, 1, 5],
                  'kernel': ['linear', 'rbf']}

svm_grid_search = GridSearchCV(SVC(), svm_param_grid, cv=5, n_jobs=-1)
svm_grid_search.fit(X_val_vec, y_val)

print("\nSVM Best Parameters:", svm_grid_search.best_params_)
print("SVM Best Score:", svm_grid_search.best_score_)


SVM Best Parameters: {'C': 5, 'kernel': 'rbf'}
SVM Best Score: 0.9835863219349459


**Fitting the best hyperparameters values and checking the performance of the model**

In [ ]:
best_svm = SVC(C=svm_grid_search.best_params_['C'],
               kernel=svm_grid_search.best_params_['kernel'])
best_svm.fit(X_train_vec, y_train)
svm_test_predictions = best_svm.predict(X_test_vec)
svm_test_accuracy = accuracy_score(y_test, svm_test_predictions)
svm_test_precision = precision_score(y_test, svm_test_predictions)
svm_test_recall = recall_score(y_test, svm_test_predictions)

print("\nSVM Validation Metrics:")
print("Accuracy:", svm_test_accuracy)
print("Precision:", svm_test_precision)
print("Recall:", svm_test_recall)
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, svm_test_predictions)
print("\nConfusion Matrix:")
print(conf_matrix)


SVM Validation Metrics:
Accuracy: 0.9872029250457038
Precision: 0.9826388888888888
Recall: 0.9929824561403509

Confusion Matrix:
[[257   5]
 [  2 283]]


In [19]:
with mlflow.start_run():
    best_svm = SVC(C=svm_grid_search.best_params_['C'],
               kernel=svm_grid_search.best_params_['kernel'])
    best_svm.fit(X_train_vec, y_train)
    

    predicted_ = best_svm.predict(X_test_vec)

    aucpr_val = aucpr_socre(y_test, predicted_)

    print("SVM Classifier model (Area under Precision_Recall Curve={:f}):".format(aucpr))

    mlflow.log_param("Area under Precision Recall Curve", aucpr_val)


    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # Model regi
    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(best_svm, "model", registered_model_name="SVM Classifier")
    else:
        mlflow.sklearn.log_model(best_svm, "model")
mlflow.end_run()

SVM Classifier model (Area under Precision_Recall Curve=0.987939):


C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2288.0_x64__qbz5n2kfra8p0\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2288.0_x64__qbz5n2kfra8p0\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


**From the Above 3 models, the performance of Support Vector Machine is better than Random Forest and Logistic Regression**